In [1]:
import os, sys

import numpy as np
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py

import powerbox
import importlib

importlib.reload(powerbox)

<module 'powerbox' from '/global/scratch/projects/pc_heptheory/fosterjw/DM21cm/powerbox/src/powerbox/__init__.py'>

In [2]:
# 21cmFAST
import py21cmfast as p21c
from py21cmfast import plotting, cache_tools
print(f'Using 21cmFAST version {p21c.__version__}')

import logging
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

Using 21cmFAST version 3.2.0


/global/scratch/projects/pc_heptheory/fosterjw/miniconda3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/global/scratch/projects/pc_heptheory/fosterjw/miniconda3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


# Configure the Run

In [3]:
N_THREADS = 40

def get_z_arr(z_start=None, z_end=5):
    
    if z_start is None:
        z_start = p21c.global_params.Z_HEAT_MAX
    z_arr = [z_end]
    while np.max(z_arr) < z_start:
        z_prev = (1 + np.max(z_arr)) * p21c.global_params.ZPRIME_STEP_FACTOR - 1
        z_arr.append(z_prev)
    return np.array(z_arr[::-1][1:])


CACHE_DIR_BASE = './cache/'

# run config
RUN_NAME = 'emf_comp_base'
f_scheme = 'CK'
run_mode = ''

p21c.config['direc'] = f'{CACHE_DIR_BASE}/{RUN_NAME}'
os.makedirs(p21c.config['direc'], exist_ok=True)
p21c.cache_tools.clear_cache()
p21c.UserParams(N_THREADS=40)

2023-04-14 06:10:07,739 | INFO | Removing BrightnessTemp_c8de8e801f54b297b161263d70867a67_r54321.h5
2023-04-14 06:10:07,744 | INFO | Removing PerturbedField_7394473c46b7cf1401e060e4535e0261_r54321.h5
2023-04-14 06:10:07,749 | INFO | Removing PerturbedField_1d6721b0b8bffce37306d166a0a73978_r54321.h5
2023-04-14 06:10:07,755 | INFO | Removing BrightnessTemp_249f7643dc486f46fa958c6e5eca2c58_r54321.h5
2023-04-14 06:10:07,762 | INFO | Removing BrightnessTemp_2e8f14f84933a93a958ef1c3d3b48be6_r54321.h5
2023-04-14 06:10:07,767 | INFO | Removing PerturbedField_63b54f956f2556220a43aaef0da72911_r54321.h5
2023-04-14 06:10:07,774 | INFO | Removing IonizedBox_c113b320805cb42a299566203c1999e8_r54321.h5
2023-04-14 06:10:07,781 | INFO | Removing IonizedBox_a8d7d61393da770cf620c054ee40a2a4_r54321.h5
2023-04-14 06:10:07,788 | INFO | Removing TsBox_01ce73ec23422b80c6b3564c4f330b70_r54321.h5
2023-04-14 06:10:07,795 | INFO | Removing BrightnessTemp_59c75c17b7ec9ab1b7d5a15ccdfec5cb_r54321.h5
2023-04-14 06:10:

2023-04-14 06:10:08,210 | INFO | Removing TsBox_5f942882bb77a83c01973b7003abd7ab_r54321.h5
2023-04-14 06:10:08,213 | INFO | Removing PerturbedField_3210ba83dd9ac8df196857e6d7d3c34c_r54321.h5
2023-04-14 06:10:08,217 | INFO | Removing PerturbedField_33c1025642a3a2a6c69ad703d466be84_r54321.h5
2023-04-14 06:10:08,222 | INFO | Removing TsBox_4ee4a0c6db8c6ded1eb7726f5486b983_r54321.h5
2023-04-14 06:10:08,228 | INFO | Removing IonizedBox_11f5e322988d6c3f1778e0a3e16a8860_r54321.h5
2023-04-14 06:10:08,234 | INFO | Removing TsBox_b72faab23dfd2e52760e700b076c62ba_r54321.h5
2023-04-14 06:10:08,240 | INFO | Removing BrightnessTemp_bc4d68d7f5c10eaf8e04a17acc5ffdd3_r54321.h5
2023-04-14 06:10:08,245 | INFO | Removing TsBox_a906b8ea7b1ab6e0da619f597554223d_r54321.h5
2023-04-14 06:10:08,251 | INFO | Removing IonizedBox_c87ad5512134b6f52ac6a9d04ab79cdd_r54321.h5
2023-04-14 06:10:08,256 | INFO | Removing BrightnessTemp_c31614f28bde89b2863e870ef07e5893_r54321.h5
2023-04-14 06:10:08,262 | INFO | Removing Br

UserParams(BOX_LEN:300, DIM:600, FAST_FCOLL_TABLES:False, HII_DIM:200, HMF:1, MINIMIZE_MEMORY:False, NO_RNG:False, N_THREADS:40, PERTURB_ON_HIGH_RES:False, POWER_SPECTRUM:0, USE_2LPT:True, USE_FFTW_WISDOM:False, USE_INTERPOLATION_TABLES:True, USE_RELATIVE_VELOCITIES:False)

# Here we trick 21cmFAST to manually evaluate a coeval box

#### The Initial Conditions

In [4]:
# Defining the range of z to operate over
z_eval = p21c.wrapper._logscroll_redshifts(20., p21c.global_params.ZPRIME_STEP_FACTOR,
                                            p21c.global_params.Z_HEAT_MAX, )

#z_eval = np.minimum(z_eval,  p21c.global_params.Z_HEAT_MAX)
cosmo_params_EMF = dict(OMm=0.32, OMb=0.049, POWER_INDEX=0.96, SIGMA_8=0.83, hlittle=0.67)

initial_conditions = p21c.initial_conditions(
    user_params = p21c.UserParams(
        HII_DIM=100, # [1]
        BOX_LEN=400, # [p-Mpc]
        N_THREADS=N_THREADS
    ),
    cosmo_params = p21c.CosmoParams(**cosmo_params_EMF),
    random_seed=54321, write=True
)

#### The 21cmFAST Run

In [5]:
mean_brightness_temp = np.zeros_like(z_eval)

for i, z in enumerate(z_eval):
    perturbed_field = p21c.perturb_field(redshift = z, init_boxes = initial_conditions, )
    spin_temp = p21c.spin_temperature(perturbed_field = perturbed_field, )
    ionized_field = p21c.ionize_box(spin_temp = spin_temp)
    
    brightness_temp = p21c.brightness_temperature(spin_temp = spin_temp, 
                                                  ionized_box=ionized_field,
                                                  perturbed_field=perturbed_field)
    mean_brightness_temp[i] = np.mean(brightness_temp.brightness_temp)
    coeval = p21c.Coeval(z, initial_conditions, perturbed_field, ionized_field, brightness_temp, spin_temp,)

    
    print(i, z, mean_brightness_temp[i])

0 35.56148918656709 -7.991377353668213
1 34.84459791200837 -7.387948513031006
2 34.14176331596501 -6.825335502624512
3 33.45270977754735 -6.251406669616699
4 32.777167080191845 -5.669429779052734
5 32.11487030569399 -5.138902187347412
6 31.465559730319022 -4.656007766723633
7 30.82898072294966 -4.217034339904785
8 30.20488364523096 -3.8190579414367676
9 29.59302375367312 -3.4594171047210693
10 28.993161103673792 -3.1353840827941895
11 28.405060455422362 -2.826450824737549
12 27.82849118164921 -2.527686834335327
13 27.26322717718381 -2.2760415077209473
14 26.70904677028621 -2.0767245292663574
15 26.1657326357171 -1.9392215013504028
16 25.63307170951238 -1.8796662092208862
17 25.11085510542884 -1.925730586051941
18 24.59887803302813 -2.1095376014709473
19 24.096939717366926 -2.484135866165161
20 23.604843320261832 -3.12617826461792
21 23.12239586309806 -4.139553070068359
22 22.64940815115169 -5.631102561950684
23 22.185694699395803 -7.77738094329834
24 21.7310736597614 -10.75320720672607

In [ ]:
lightcone = p21c.run_lightcone(redshift = ionized_field.redshift,
                               user_params = ionized_field.user_params,
                               cosmo_params = ionized_field.cosmo_params,
                               astro_params = ionized_field.astro_params,
                               flag_options=ionized_field.flag_options)

/global/scratch/projects/pc_heptheory/fosterjw/miniconda3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/fosterjw/miniconda3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/fosterjw/miniconda3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/global/scratch/projects/pc_heptheory/fosterjw/miniconda3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vx_2LP

In [ ]:
fig, ax = plt.subplots(figsize = (14, 10))
plotting.lightcone_sliceplot(lightcone, fig =fig,ax = ax) 

In [ ]:
def compute_power(
   box,
   length,
   n_psbins,
   log_bins=True,
   ignore_kperp_zero=True,
   ignore_kpar_zero=False,
   ignore_k_zero=False,
):
    # Determine the weighting function required from ignoring k's.
    k_weights = np.ones(box.shape, int)
    n0 = k_weights.shape[0]
    n1 = k_weights.shape[-1]

    if ignore_kperp_zero:
        k_weights[n0 // 2, n0 // 2, :] = 0
    if ignore_kpar_zero:
        k_weights[:, :, n1 // 2] = 0
    if ignore_k_zero:
        k_weights[n0 // 2, n0 // 2, n1 // 2] = 0

    res = powerbox.tools.get_power(
        box,
        boxlength=length,
        bins=n_psbins,
        bin_ave=False,
        get_variance=False,
        log_bins=log_bins,
        k_weights=k_weights,
    )

    res = list(res)
    k = res[1]
    if log_bins:
        k = np.exp((np.log(k[1:]) + np.log(k[:-1])) / 2)
    else:
        k = (k[1:] + k[:-1]) / 2

    res[1] = k
    return res

def powerspectra(brightness_temp, n_psbins=50, nchunks=20, min_k=0.1, max_k=1.0, logk=True):
    data = []
    chunk_indices = list(range(0,brightness_temp.n_slices,round(brightness_temp.n_slices / nchunks),))    
    
    if len(chunk_indices) > nchunks:
        chunk_indices = chunk_indices[:-1]
    chunk_indices.append(brightness_temp.n_slices)

    for i in range(nchunks):
        start = chunk_indices[i]
        end = chunk_indices[i + 1]
        chunklen = (end - start) * brightness_temp.cell_size

        power, k = compute_power(
            brightness_temp.brightness_temp[:, :, start:end],
            (BOX_LEN, BOX_LEN, chunklen),
            n_psbins,
            log_bins=logk,
        )
        data.append({"k": k, "delta": power * k ** 3 / (2 * np.pi ** 2)})
    return data

In [ ]:
BOX_LEN = 400
HII_DIM = 100


k_fundamental = 2*np.pi/BOX_LEN
k_max = k_fundamental * HII_DIM
Nk=np.floor(HII_DIM/1).astype(int)

In [ ]:
out = powerspectra(lightcone, min_k = k_fundamental, max_k = k_max)

In [ ]:
fig, axs = plt.subplots(ncols = 5, nrows = 4, figsize = (30, 16))

for i, item in enumerate(out):
    row_index, col_index = np.unravel_index(i, axs.shape)
    
    ax = axs[row_index, col_index]
    ax.plot(item['k'], item['delta'], color = 'black')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.text(.6, .1, 'Redshift Chunk ' + str(i), transform=ax.transAxes, fontsize = 18)
    
for i in range(axs.shape[0]-1):
    for j in range(axs.shape[1]):
        axs[i, j].set_xticklabels([])

for j in range(axs.shape[1]):
    axs[-1, j].set_xlabel('k [Mpc$^{-1}$]', fontsize = 22)
for i in range(axs.shape[0]):
    axs[i, 0].set_ylabel('$k^3 P(k)$', fontsize = 22)
    
plt.tight_layout()
plt.show()